In [10]:
import panel
import gempy as gp
# import gempy_viewer as gpv
import pickle
import vtk
import rasterio
import pyevtk
from discretize import TensorMesh
from discretize.utils import mkvc, active_from_xyz
from scipy.interpolate import LinearNDInterpolator
from SimPEG import maps
from SimPEG.potential_fields import gravity
from SimPEG import discretize
from matplotlib.colors import LogNorm, SymLogNorm

# from gempy.core.data import GeoModel
from gempy.core.data import Grid
# from gempy.core.data.grid_modules import RegularGrid
# from PyQt6 import QtWidgets, QtGui

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, time
import pyvista as pv
from gempy_engine.core.data.stack_relation_type import StackRelationType


import SimPEG as simpeg

from pymatsolver import Solver

np.random.seed(55500)

In [11]:
geo_model: gp.data.GeoModel = gp.create_geomodel(
    project_name='Model1',
    extent=[853610.2891, 855569.4948, 
            987530.5888, 988620.4712, 
            -6600, 3000], #x, y, z
    resolution=None,
    refinement=4, # We will use octrees
    importer_helper= gp.data.ImporterHelper(
        path_to_orientations= "capas_model_orientaciones.csv",
        path_to_surface_points= "capas_model_points.csv",

    )
)

AttributeError: module 'gempy' has no attribute 'create_geomodel'

In [ ]:
gp.map_stack_to_surfaces(
    gempy_model=geo_model,
    mapping_object=  # TODO: This mapping I do not like it too much. We should be able to do it passing the data objects directly
    {
        "Strat_Series1": ('Complejo_Dacitico'),     
        "Strat_Series2": ('Cuerpo_Intrusivo'),
        "Strat_Series3": ('Flujo_Piroclastico'),
        "Strat_Series4": ('Sucesion_Piroclastica'),
        "Strat_Series5": ('Sucesion_Volcanica')
    }
)
geo_model.structural_frame  # Display the resulting structural frame


In [ ]:
plot = gpv.plot_2d(geo_model, show_lith=False, show_boundaries=False,)

In [ ]:
gp.set_topography_from_file(
    grid=geo_model.grid,
    filepath=os.path.join("Volcan_CM2.tif"),
)

gpv.plot_2d(geo_model, show_topography=True, section_names=['topography'], show_data=True)

In [ ]:
geo_model.grid

In [ ]:
grid = Grid()

In [ ]:
grav_res = 30 #Resolución(numero de puntos a lo largo del eje)
X = np.linspace(853610.2891, 855569.4948, grav_res)
Y = np.linspace(987530.5888, 988620.4712, grav_res)
Z = 2300
xyz = np.meshgrid(X, Y, Z)
xy_ravel = np.vstack(list(map(np.ravel, xyz))).T #Se aplana la malla 3D a una matriz 2D, resultando en una lista de matrices 
                                                 # 1D aplanadas que se apilan para formar una única matriz 2D de 2 columnas (X,Y)
xy_ravel

In [ ]:
gpv.plot_2d(geo_model, direction='z', show=True, topography=True)
plt.scatter(xy_ravel[:, 0], xy_ravel[:, 1], s=2) #plotea las columnas X y Y del xy_ravel
plt.show()

In [ ]:
gp.set_centered_grid(
    grid=geo_model.grid,
    centers=xy_ravel, #Define las coordenadas centrales de cada celda de la cuadrícula.
    resolution=np.array([9, 9, 32]), #Define el tamaño (metros) de las celdas de la nueva malla.
    radius=np.array([4500, 4500, 4500]) #Define el radio (metros) de influencia de cada celda.
)

In [2]:
geo_model.grid.centered_grid.kernel_grid_centers   #coordenadas centrales de las celdas en un tipo específico de cuadrícula
                                                   #llamada "cuadrícula centrada". 

NameError: name 'geo_model' is not defined

In [ ]:
gravity_gradient = gp.calculate_gravity_gradient(geo_model.grid.centered_grid) #Opera para calcular el tensor del gradiente de 
                                                                               #gravedad en Gempy
gravity_gradient

In [ ]:
geo_model.geophysics_input = gp.data.GeophysicsInput(
    tz=gravity_gradient,
    densities=np.array([-0.2, 0, 0.1, 0.2, 0.3, 0.5]),
)

In [ ]:
geo_model.interpolation_options.mesh_extraction = True
sol = gp.compute_model(
    gempy_model=geo_model,
    engine_config=gp.data.GemPyEngineConfig(
        backend=gp.data.AvailableBackends.numpy,
        dtype='float32'
    )
)

grav = sol.gravity

In [ ]:
gpv.plot_2d(geo_model, 
            cell_number=['mid'],
            direction='x',
            show_topography=True, 
            figsize=(12, 17), 
            show_block=False, 
            show_values=False, 
            n_axis=2, 
            legend=True,
            show_data=False, 
            ve=0.10))

In [ ]:
gpv.plot_2d(geo_model, cell_number=[-1], direction=['z'], show=True, show_results=False, show_data=False, legend=False, topography=True,
            kwargs_regular_grid={'alpha': .5})

plt.scatter(xy_ravel[:, 0], xy_ravel[:, 1], s=1)
plt.imshow(grav.reshape(grav_res, grav_res),
           extent=(xy_ravel[:, 0].min() + (xy_ravel[0, 0] - xy_ravel[1, 0]) / 2,
                   xy_ravel[:, 0].max() - (xy_ravel[0, 0] - xy_ravel[1, 0]) / 2,
                   xy_ravel[:, 1].min() + (xy_ravel[0, 1] - xy_ravel[30, 1]) / 2,
                   xy_ravel[:, 1].max() - (xy_ravel[0, 1] - xy_ravel[30, 1]) / 2),
           cmap='viridis_r', origin='lower', alpha=.8)
cbar = plt.colorbar()
cbar.set_label(r'$\mu$gal')
plt.show()

# sphinx_gallery_thumbnail_number = -2

In [ ]:
dx = 35
ncx = 40
dy = 40
ncy = 10
dz = 200
ncz = 40
dpadh = 3.5
exp_h = 1.5

hx = [(dx, dpadh, -exp_h), (dx, ncx), (dx, dpadh, exp_h)]
hy = [(dy, dpadh, -exp_h), (dy, ncy), (dy, dpadh, exp_h)]
hz = [(dz, 3, -exp_h), (dz, ncz)]
mesh = TensorMesh([hx, hy, hz], x0=[853614,987543,-6000])
mesh.plot_grid(color="midnightblue", linewidth=0.1)

In [ ]:
# mesh.plot_grid